## Main File To Run Code

## Imports

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
import pandas as pd
from joblib import load
from flask import Flask, request, jsonify

## 1 Connecting to Firebase Account

In [4]:
cred = credentials.Certificate('./firebase-sdk.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

## 2 Create Classes

In [5]:
feature_order = ['Name', 'Gender', 'Age', 'CountryofBirth', 'Smoking', 'Alcohol', 'FamilyHistory', 'CloseContact', 'HealthConditions', 'OccupationalRisks', 'PhysicalActivity', 'Diet', 'AirPollution', 'LongTermCough', 'ChestPain', 'AppetiteLoss', 'WeightLoss', 'Chills', 'Fatigue', 'NightSweat', 'CoughBlood', 'Fevers', 'NitricOxide']

## 3 Predict User's Data

In [6]:
# Function to fetch a user's data from Firestore
def fetch_user_data(user_id):
    doc_ref = db.collection('users').document(user_id)
    doc = doc_ref.get()
    if doc.exists:
        #print(doc.to_dict())
        return doc.to_dict()
    else:
        print("No such document!")
        return None

# Function to preprocess user's data
def preprocess_user_data(user_data):
    # Assuming user_data is a dictionary containing the user's features
    df = pd.DataFrame([user_data])  # Convert dict to DataFrame
    df = df[feature_order]
    # Apply the same preprocessing as your training data
    # Load label encoder and transform 'Country of Birth'
    le_country = load('label_encoder.joblib')
    df['encoded_country'] = le_country.transform(df['CountryofBirth'])
    
    # Ensure all preprocessing steps match training, including dropping unused columns
    processed_df = df.drop(['Name', 'CountryofBirth'], axis=1)
    
    # Return preprocessed user data ready for prediction
    return processed_df

# Load your trained Decision Tree model
classifier = load('classifier.joblib')

# Predict function for a user
def predict_tbpred(user_id):
    user_data = fetch_user_data(user_id)
    if user_data:
        processed_data = preprocess_user_data(user_data)
        prediction = classifier.predict(processed_data)
        return prediction
    else:
        return "User data not found"


# Example usage
user_id = '4PzkRocqvKdL71AtKXiu'
prediction = predict_tbpred(user_id)
print(f"Predicted tb_pred for user {user_id}: {prediction}")

Predicted tb_pred for user 4PzkRocqvKdL71AtKXiu: [1]
